In [1]:
import torchaudio
torchaudio.set_audio_backend("soundfile")
from agents.tester import Tester
from utils.config import *


# YAML = 'configs/pre_trained/ResNet18.yaml'
# YAML = 'configs/pre_trained/ResNet34.yaml'
# YAML = 'configs/pre_trained/ResNetSE34L.yaml'
# YAML = 'configs/pre_trained/ResNetSE34V2.yaml'
# YAML = 'configs/pre_trained/VGGVox.yaml'
# YAML = 'configs/pre_trained/AutoSpeech.yaml'
# YAML = 'configs/pre_trained/Cnn3D.yaml'
# YAML = 'configs/pre_trained/SincNet.yaml'
YAML = 'configs/pre_trained/MobileNet.yaml'

config = process_config(YAML)

agent = Tester(config)
agent.run()

[INFO]: Operation will be on ***** GPU-CUDA ***** 
[INFO]: __Python VERSION:  3.9.5 | packaged by conda-forge | (default, Jun 19 2021, 00:22:33) [MSC v.1916 64 bit (AMD64)]
[INFO]: __pyTorch VERSION:  1.9.0
[INFO]: __CUDA VERSION
[INFO]: __CUDNN VERSION:  7605
[INFO]: __Number CUDA Devices:  1
[INFO]: __Devices
[INFO]: Active CUDA Device: GPU 0
[INFO]: Available devices  1
[INFO]: Current cuda device  0
[INFO]: Computing metrics...
C:\Users\giang\anaconda3\envs\thesis\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Warning! No positional inputs found for a module, assuming batch size is 1.


[INFO]: ================================================================
[INFO]: MobileNetV2
[INFO]: ================================================================
[INFO]: Total params: 2193128
[INFO]: Trainable params: 2193128
[INFO]: Non-trainable params: 0
[INFO]: ----------------------------------------------------------------
[INFO]: Average flops cost: 4.25 GMac
[INFO]: ----------------------------------------------------------------
[INFO]: Average inference time for every batch size:
[INFO]: 1: 20.79 +/- 0.98
[INFO]: 2: 19.39 +/- 0.62
[INFO]: 4: 18.18 +/- 0.19
[INFO]: 8: 17.71 +/- 0.12
[INFO]: 16: 17.69 +/- 0.52
[INFO]: 32: 17.31 +/- 0.04
[INFO]: 64: 25.36 +/- 0.04
[INFO]: ----------------------------------------------------------------
[INFO]: Memory usage for every batch size:
[INFO]: 1:  0.0292
[INFO]: 2:  0.0480
[INFO]: 4:  0.0877
[INFO]: 8:  0.1670
[INFO]: 16:  0.3261
[INFO]: 32:  0.6439
[INFO]: 64:  1.2767
[INFO]: ========================================================